# Hierarchical Indexing

Today's lecture is based on Jake VanderPlas' *Python Data Science Handbook*
- https://jakevdp.github.io/PythonDataScienceHandbook/03.05-hierarchical-indexing.html
- https://jakevdp.github.io/PythonDataScienceHandbook/03.06-concat-and-append.html
- https://jakevdp.github.io/PythonDataScienceHandbook/03.07-merge-and-join.html
- https://jakevdp.github.io/PythonDataScienceHandbook/03.08-aggregation-and-grouping.html


## Higher Dimensional Data
- Data indexed by more than one or two keys
- Also called ``multi-indexing``
- Incorporates mulitple index levels within a single index
  - This allows for presenting high-dimensional data in one-dimensional ``Series`` and two-dimensional ``DataFrame`` objects.

In [1]:
%%HTML
<style type="text/css">
    table.dataframe td, table.dataframe th {
        border-style: dotted;
    }
</style>

## ``MultipleIndex`` objects

In [2]:
import pandas as pd
import numpy as np

### How to represent two-dimensional data within a one-dimensional ``Series``?

#### The bad way - using Python tuples as keys

In [3]:
index = [('California', 2000), ('California', 2010),
         ('New York', 2000), ('New York', 2010),
         ('Texas', 2000), ('Texas', 2010)]
populations = [33871648, 37253956,
               18976457, 19378102,
               20851820, 25145561]
pop = pd.Series(populations, index=index)
pop

(California, 2000)    33871648
(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
(Texas, 2010)         25145561
dtype: int64

In [4]:
pop[('California', 2010):('Texas', 2000)]

(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
dtype: int64

Pros:
- Indexing and slicing are straight foward

Cons:
- Run into a complexity if you need to select all values using one of the tuple elements (e.g. all values from 2010).

In [5]:
pop[[i for i in pop.index if i[1] == 2010]]

(California, 2010)    37253956
(New York, 2010)      19378102
(Texas, 2010)         25145561
dtype: int64

#### Using ``MultiIndex``
Pandas ``MultiIndex`` provides the same operations as manually creating multi-index from the tuples

In [6]:
index = pd.MultiIndex.from_tuples(index)
index

MultiIndex([('California', 2000),
            ('California', 2010),
            (  'New York', 2000),
            (  'New York', 2010),
            (     'Texas', 2000),
            (     'Texas', 2010)],
           )

``MultiIndex`` contains multiple levels of indexing (the state names, and the years)

[Re-index](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.reindex.html) the ``Series`` to get the hierarchical representation of the data.
- The first two columns show the multiple index values. 
- Missing entries in the first column indicates the same values as the line above it. 

In [7]:
pop = pop.reindex(index)
pop

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

#### Accessing all data for year 2010
- The results is a single-indexed array with only the specified key.
- This allows simpler sytanx and much more efficient operations.

In [8]:
pop[:, 2010]

California    37253956
New York      19378102
Texas         25145561
dtype: int64

### MultiIndex as extra dimension
- [unstack()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.unstack.html) (a.k.a. pivot) method converts multi indexed ``Series`` into a conventionally indexed ``DataFrame``

NOTE: Refer to the official [Pandas documentation](https://pandas.pydata.org/pandas-docs/version/0.25.3/user_guide/advanced.html) for more details on MultiIndex.

In [9]:
pop_df = pop.unstack()
pop_df

,2000,2010
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


- [stack()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.stack.html) provides the opposite operation

>Return a reshaped DataFrame or Series having a multi-level index with one or more new inner-most levels compared to the current DataFrame. The new inner-most levels are created by pivoting the columns of the current dataframe:

> if the columns have a single level, the output is a Series;

> if the columns have multiple levels, the new index level(s) is (are) taken from the prescribed level(s) and the output is a DataFrame.



In [10]:
pop_df.stack()

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

Why bother with hierarchical indexing if we can use ``DataFrames``?
>The reason is simple: just as we were able to use multi-indexing to represent two-dimensional data within a one-dimensional Series, we can also use it to represent data of three or more dimensions in a Series or DataFrame. Each extra level in a multi-index represents an extra dimension of data; taking advantage of this property gives us much more flexibility in the types of data we can represent. 

In [11]:
pop_df = pd.DataFrame({'total': pop,
                       'under18': [9267089, 9284094,
                                   4687374, 4318033,
                                   5906301, 6879014]})
pop_df

total  under18
California 2000  33871648  9267089
           2010  37253956  9284094
New York   2000  18976457  4687374
           2010  19378102  4318033
Texas      2000  20851820  5906301
           2010  25145561  6879014

In [12]:
pop_df['total']

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
Name: total, dtype: int64

In [13]:
pop_df['under18']

California  2000    9267089
            2010    9284094
New York    2000    4687374
            2010    4318033
Texas       2000    5906301
            2010    6879014
Name: under18, dtype: int64

Universal functions (ufuncs) work with hierarchical indices as expected. 

In [14]:
f_u18 = pop_df['under18'] / pop_df['total']
f_u18

California  2000    0.273594
            2010    0.249211
New York    2000    0.247010
            2010    0.222831
Texas       2000    0.283251
            2010    0.273568
dtype: float64

In [15]:
f_u18.unstack()

,2000,2010
California,0.273594,0.249211
New York,0.247010,0.222831
Texas,0.283251,0.273568


### Methods of MultiIndex Creation

#### Pass a list of two or more index arrays to the constructor

In [16]:
df = pd.DataFrame(np.random.rand(4, 2),
                  index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                  columns=['data1', 'data2'])
df

data1     data2
a 1  0.796714  0.873359
  2  0.241682  0.297875
b 1  0.382280  0.029880
  2  0.861464  0.462420

#### Pass a dictionary with tuples as keys - Pandas will use a MultiIndex automatically.

In [17]:
data = {('California', 2000): 33871648,
        ('California', 2010): 37253956,
        ('Texas', 2000): 20851820,
        ('Texas', 2010): 25145561,
        ('New York', 2000): 18976457,
        ('New York', 2010): 19378102}
pd.Series(data)

California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
New York    2000    18976457
            2010    19378102
dtype: int64

#### Explicitly create a ``MultiIndex``

Using class method constructors available in pd.MultiIndex:

From a single list of arrays.

In [18]:
pd.MultiIndex.from_arrays([['a', 'a', 'b', 'b'], [1, 2, 1, 2]])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

From a list of tuples

In [19]:
pd.MultiIndex.from_tuples([('a', 1), ('a', 2), ('b', 1), ('b', 2)])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

From a Cartesian product of single indices

In [20]:
pd.MultiIndex.from_product([['a', 'b'], [1, 2]])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

Directly using `MultiIndex`'s internal encoding by passing *levels* (a list of list containing available index values for each level) and *labels* (a list of list referencing labels).

Note:
>Changed in version 0.24.0: MultiIndex.labels has been renamed to MultiIndex.codes and MultiIndex.set_labels to MultiIndex.set_codes.

In [21]:
pd.MultiIndex(levels=[['a', 'b'], [1, 2]],
              labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

TypeError: __new__() got an unexpected keyword argument 'labels'

Refer to https://stackoverflow.com/questions/52064543/how-labels-are-maps-in-pandas-multiindex for more information on *level* and *labels*.

In [22]:
mi = pd.MultiIndex(levels=[['a', 'b'], [1, 2]],
              codes=[[0, 0, 1, 1], [0, 1, 0, 1]])

In [23]:
mi

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

In [24]:
# nlevels attribute indicates an integer number of levels in this MultiIndex.
mi.nlevels

2

Note: 
- *levels*: The unique labesl for each level
- *codes*: Integers for each level designation which labels at each location

Refer to pd.MultiIndex [API documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.MultiIndex.html) for the list of parameters.

### MultiIndex level names

You can assign names to the levels of the *MultiIndex* by passing *names* argument to any of the ``MultiIndex`` constructor, or by setting the *names* attribute of the index.
- Useful way to keep track of the meaning of various index values. 

In [25]:
pop

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [26]:
pop.index.names=['state', 'year']
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

### MultiIndex for columns

**In a *DataFrame*, the rows and columns are completely symmetric**
- Just as the rows can have multiple levels of indices, the columns can have multiple levels as well.

In [27]:
# hierarchical indices and columns
index = pd.MultiIndex.from_product([[2013, 2014], [1, 2]],
                                   names=['year', 'visit'])
columns = pd.MultiIndex.from_product([['Bob', 'Guido', 'Sue'], ['HR', 'Temp']],
                                     names=['subject', 'type'])

In [28]:
# mock some data
data = np.round(np.random.randn(4, 6), 1)
data[:, ::2] *= 10
data += 37
data

array([[32. , 39.5, 37. , 36.1, 26. , 38.6],
       [37. , 36.8, 32. , 37. , 36. , 37. ],
       [43. , 37.7, 24. , 35.5, 42. , 36. ],
       [39. , 36.1, 55. , 36.7, 41. , 37.9]])

In [29]:
# create the DataFrame
health_data = pd.DataFrame(data, index=index, columns=columns)
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      32.0  39.5  37.0  36.1  26.0  38.6
     2      37.0  36.8  32.0  37.0  36.0  37.0
2014 1      43.0  37.7  24.0  35.5  42.0  36.0
     2      39.0  36.1  55.0  36.7  41.0  37.9

The above example is an example of four-dimensional data. Dimensions include the subject, the measurement type, the year and the visit number.  

>For complicated records containing multiple labeled measurements across multiple times for many subjects (people, countries, cities, etc.) use of hierarchical rows and columns can be extremely convenient!

We can index the top-level column by person's name and get a full ``DataFrame`` containing just that person's information.

In [30]:
health_data['Guido']

type          HR  Temp
year visit            
2013 1      37.0  36.1
     2      32.0  37.0
2014 1      24.0  35.5
     2      55.0  36.7

### Indexing and Slicing a MultiIndex

In [31]:
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

#### Accessing single element by indexing with multiple terms

In [32]:
pop['California', 2000]

33871648

#### *Partial indexing*, indexing just one of the levels in the index. 
- The result is another ``Series`` with the lower-level indices maintained:

In [33]:
pop['California']

year
2000    33871648
2010    37253956
dtype: int64

#### Partial slicing
Note: This requires the MultiIndex to be sorted. 

In [34]:
pop.loc['California':'New York']

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
dtype: int64

With sorted, indices, partial indexing can be performed on lower levels

In [35]:
pop[:, 2000]

state
California    33871648
New York      18976457
Texas         20851820
dtype: int64

#### Selection based on Boolean masks

In [36]:
pop[pop > 22000000]

state       year
California  2000    33871648
            2010    37253956
Texas       2010    25145561
dtype: int64

#### Selection based on fancy indexing

In [37]:
pop[['California', 'Texas']]

state       year
California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
dtype: int64

## Multiply indexed DataFrames

In [38]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      32.0  39.5  37.0  36.1  26.0  38.6
     2      37.0  36.8  32.0  37.0  36.0  37.0
2014 1      43.0  37.7  24.0  35.5  42.0  36.0
     2      39.0  36.1  55.0  36.7  41.0  37.9

#### Using column names

In [39]:
health_data['Guido', 'HR']

year  visit
2013  1        37.0
      2        32.0
2014  1        24.0
      2        55.0
Name: (Guido, HR), dtype: float64

In [40]:
health_data.loc[2013,1]

subject  type
Bob      HR      32.0
         Temp    39.5
Guido    HR      37.0
         Temp    36.1
Sue      HR      26.0
         Temp    38.6
Name: (2013, 1), dtype: float64

#### Using ``loc``, ``iloc``, and ``ix`` indexers.

In [41]:
health_data.iloc[:2, :2]

subject      Bob      
type          HR  Temp
year visit            
2013 1      32.0  39.5
     2      37.0  36.8

#### Passing in a tuple of multiple indices to ``loc`` or ``iloc`

In [42]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      32.0  39.5  37.0  36.1  26.0  38.6
     2      37.0  36.8  32.0  37.0  36.0  37.0
2014 1      43.0  37.7  24.0  35.5  42.0  36.0
     2      39.0  36.1  55.0  36.7  41.0  37.9

In [43]:
health_data.loc[:, ('Bob', 'HR')]

year  visit
2013  1        32.0
      2        37.0
2014  1        43.0
      2        39.0
Name: (Bob, HR), dtype: float64

#### Using IndexSlice object

In [44]:
idx = pd.IndexSlice
health_data.loc[idx[:, 1], idx[:, 'HR']]

,subject,Bob,Guido,Sue
,type,HR,HR,HR
year,visit,,,
2013,1,32.0,37.0,26.0
2014,1,43.0,24.0,42.0


## Rearranging Multi-Indices

### Sorted and unsorted indices
- **Many of the MultiIndex slicing operations will fail if the index is not sorted.**

In [45]:
index = pd.MultiIndex.from_product([['a', 'c', 'b'], [1, 2]])
data = pd.Series(np.random.rand(6), index=index)
data.index.names = ['char', 'int']
data

char  int
a     1      0.221740
      2      0.073330
c     1      0.616179
      2      0.998256
b     1      0.982140
      2      0.689485
dtype: float64

In [46]:
try:
    data['a':'b']
except KeyError as e:
    # This is the result of the MultiIndex not being sorted.
    print(type(e))
    print(e)

<class 'pandas.errors.UnsortedIndexError'>
'Key length (1) was greater than MultiIndex lexsort depth (0)'


- Partial slices and other similar operations require the levels in the MultiIndex to be sorted
- Pandas provides a number of convenience routines to perform sorting
  - [sort_index()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sort_index.html)
  - sortlevel(), **Note: Deprecated since version 0.20.0: Use DataFrame.sort_index()**

In [47]:
data = data.sort_index()
data

char  int
a     1      0.221740
      2      0.073330
b     1      0.982140
      2      0.689485
c     1      0.616179
      2      0.998256
dtype: float64

The partial slicing works as expected with sorted index. 

In [48]:
data['a':'b']

char  int
a     1      0.221740
      2      0.073330
b     1      0.982140
      2      0.689485
dtype: float64

### Stacking and unstacking indices

#### Stacking
Rotating, or pivoting, the **innermost column index** into the **innermost row index**
  - Rearranging data vertically, thus "stacking"
  - Making the shape of the dataframe **taller and narrower**  
  
![](https://pandas.pydata.org/docs/_images/reshaping_stack.png)

#### Unstacking
Converting the **innermost row index** back into the **innermost column index**.
  - Reshaping the dataframe to be **shorter and wider**
  - By default, unstack() unstacks the **last (inner most) level**. 
  
![](https://pandas.pydata.org/docs/_images/reshaping_unstack.png)  

![](https://pandas.pydata.org/docs/_images/reshaping_unstack_1.png)


**References**
- https://nikgrozev.com/2015/07/01/reshaping-in-pandas-pivot-pivot-table-stack-and-unstack-explained-with-pictures/
- https://pandas.pydata.org/docs/user_guide/reshaping.html

In [49]:
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [50]:
pop.unstack(level=0)

state,California,New York,Texas
year,,,
2000,33871648,18976457,20851820
2010,37253956,19378102,25145561


In [51]:
pop.unstack(level=1)

year,2000,2010
state,,
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


Note: *level* argument specifies level(s) of index to unstack. Level name can also be passed.

In [52]:
# By default, last (inner-most) level is unstacked.
pop.unstack()

year,2000,2010
state,,
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


In [53]:
# Recoverying original series
pop.unstack().stack()

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

#### Rearranging hierarchical data by turnning index labels into columns.
- Use [reset_index](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.reset_index.html) method.
  - Useful when the index needs to be treated as a column, or when the index is meaningless and needs to be reset to the default before another operation.
  - 'name' parameter is used for the column containing the original Series values.

In [54]:
pop_flat = pop.reset_index(name='population')
pop_flat

,state,year,population
0,California,2000,33871648
1,California,2010,37253956
2,New York,2000,18976457
3,New York,2010,19378102
4,Texas,2000,20851820
5,Texas,2010,25145561


#### Setting ``MultiIndex`` from the column values
- [set_index](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.set_index.html) method of the ``DataFrame`` returns a multiply indexed ``DataFrame``

In [55]:
indexed_pop_flat = pop_flat.set_index(['state', 'year'])

In [56]:
indexed_pop_flat

population
state      year            
California 2000    33871648
           2010    37253956
New York   2000    18976457
           2010    19378102
Texas      2000    20851820
           2010    25145561

In [57]:
indexed_pop_flat.index

MultiIndex([('California', 2000),
            ('California', 2010),
            (  'New York', 2000),
            (  'New York', 2010),
            (     'Texas', 2000),
            (     'Texas', 2010)],
           names=['state', 'year'])

## Data Aggregation on Multi-Indices
For data aggregation on hierarchically indexed data, ``level`` parameter can be passed to the aggregation methods to control the subset of the data on which the aggregation is done. 

In [58]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      32.0  39.5  37.0  36.1  26.0  38.6
     2      37.0  36.8  32.0  37.0  36.0  37.0
2014 1      43.0  37.7  24.0  35.5  42.0  36.0
     2      39.0  36.1  55.0  36.7  41.0  37.9

In [59]:
data_mean = health_data.mean(level='year')
data_mean

subject   Bob        Guido          Sue       
type       HR   Temp    HR   Temp    HR   Temp
year                                          
2013     34.5  38.15  34.5  36.55  31.0  37.80
2014     41.0  36.90  39.5  36.10  41.5  36.95

In [60]:
data_mean.mean(axis=1, level='type')

type,HR,Temp
year,,
2013,33.333333,37.50
2014,40.666667,36.65


# Combining Datasets: Concat and Append  

Combining of the data from different sources:
- Simple concatenation
- Database-style joins and merges
- Pandas includes functions and methods to allow data mangling easy and straightforward

In [61]:
def make_df(cols, ind):
    """Quickly make a DataFrame"""
    data = {c: [str(c) + str(i) for i in ind]
            for c in cols}
    return pd.DataFrame(data, ind)

# example DataFrame
make_df('ABC', range(3))

,A,B,C
0,A0,B0,C0
1,A1,B1,C1
2,A2,B2,C2


In [62]:
class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

### Simple Concatenation with pd.concat
pd.concat() can be used for a simple concatenation of Series or DataFrame objects

In [63]:
ser1 = pd.Series(['A', 'B', 'C'], index=[1, 2, 3])
ser2 = pd.Series(['D', 'E', 'F'], index=[4, 5, 6])
pd.concat([ser1, ser2])

1    A
2    B
3    C
4    D
5    E
6    F
dtype: object

#### Concatenating higher-dimension objects
- By default, [pd.concat](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html) concatenates two DataFrames row-wise (i.e., axis=0).
- ``axis`` parameter is used to specify the axis along which the concatenation will take place.

In [64]:
df1 = make_df('AB', [1, 2])
df2 = make_df('AB', [3, 4])
display('df1', 'df2', 'pd.concat([df1, df2])')

,A,B
1,A1,B1
2,A2,B2
,A,B
3,A3,B3
4,A4,B4
,A,B
1,A1,B1
2,A2,B2
3,A3,B3
4,A4,B4


In [65]:
df3 = make_df('AB', [0, 1])
df4 = make_df('CD', [0, 1])
display('df3', 'df4', "pd.concat([df3, df4], axis=1)")

df3
    A   B
0  A0  B0
1  A1  B1

df4
    C   D
0  C0  D0
1  C1  D1

pd.concat([df3, df4], axis=1)
    A   B   C   D
0  A0  B0  C0  D0
1  A1  B1  C1  D1

**Note: Unlike np.concatenate, pd.concat preserves indices, even if the result ends up with duplicate indices.**

In [66]:
x = make_df('AB', [0, 1])
y = make_df('AB', [2, 3])
y.index = x.index  # make duplicate indices!
display('x', 'y', 'pd.concat([x, y])')

,A,B
0,A0,B0
1,A1,B1
,A,B
0,A2,B2
1,A3,B3
,A,B
0,A0,B0
1,A1,B1
0,A2,B2
1,A3,B3


### Handling duplicate indices

#### verify_integrity
Setting ``verify_integrity`` parameter with ``True`` boolean value will cause an exception to be thrown when there are duplicate indices. 

In [67]:
try:
    pd.concat([x, y], verify_integrity=True)
except ValueError as e:
    print("ValueError:", e)

ValueError: Indexes have overlapping values: Int64Index([0, 1], dtype='int64')


### ignore_index

If the index doesn not matter, it can be ignored by using ``ignore_index`` flag. 
> If True, do not use the index values along the concatenation axis. The resulting axis will be labeled 0, …, n - 1. This is useful if you are concatenating objects where the concatenation axis does not have meaningful indexing information. Note the index values on the other axes are still respected in the join.

In [68]:
display('x', 'y', 'pd.concat([x, y], ignore_index=True)')

,A,B
0,A0,B0
1,A1,B1
,A,B
0,A2,B2
1,A3,B3
,A,B
0,A0,B0
1,A1,B1
2,A2,B2
3,A3,B3


### Adding MultiIndex keys
Use ``keys`` option to specify a label for the data source. This results in a hierarchically indexed series.

>If multiple levels passed, should contain tuples. Construct hierarchical index using the passed keys as the outermost level.

In [69]:
display('x', 'y', "pd.concat([x, y], keys=['x', 'y'])")

x
    A   B
0  A0  B0
1  A1  B1

y
    A   B
0  A2  B2
1  A3  B3

pd.concat([x, y], keys=['x', 'y'])
      A   B
x 0  A0  B0
  1  A1  B1
y 0  A2  B2
  1  A3  B3

### Concatenating with joins
- By default, outer join (join='outer'), or a union of the all input columns, is used. 

In [70]:
df5 = make_df('ABC', [1, 2])
df6 = make_df('BCD', [3, 4])
display('df5', 'df6', 'pd.concat([df5, df6])')

df5
    A   B   C
1  A1  B1  C1
2  A2  B2  C2

df6
    B   C   D
3  B3  C3  D3
4  B4  C4  D4

pd.concat([df5, df6])
     A   B   C    D
1   A1  B1  C1  NaN
2   A2  B2  C2  NaN
3  NaN  B3  C3   D3
4  NaN  B4  C4   D4

#### Getting an intersection of the columns using join='inner'

In [71]:
display('df5', 'df6',
        "pd.concat([df5, df6], join='inner')")

df5
    A   B   C
1  A1  B1  C1
2  A2  B2  C2

df6
    B   C   D
3  B3  C3  D3
4  B4  C4  D4

pd.concat([df5, df6], join='inner')
    B   C
1  B1  C1
2  B2  C2
3  B3  C3
4  B4  C4

### Using append() method

In [72]:
display('df1', 'df2', 'df1.append(df2)')

,A,B
1,A1,B1
2,A2,B2
,A,B
3,A3,B3
4,A4,B4
,A,B
1,A1,B1
2,A2,B2
3,A3,B3
4,A4,B4


## Merge and Join
Python's [pd.merge](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html) function provides database style, high-performance, in-memory join and merge operations. 

Categories of joins
- one-to-one
- many-to-one
- many-to-many

#### One-to-one joins

In [73]:
df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'group': ['Accounting', 'Engineering', 'Engineering', 'HR']})
df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue'],
                    'hire_date': [2004, 2008, 2012, 2014]})
display('df1', 'df2')

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
,employee,hire_date
0,Lisa,2004
1,Bob,2008
2,Jake,2012
3,Sue,2014


In [74]:
df3 = pd.merge(df1, df2)
df3

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


>The pd.merge() function recognizes that each DataFrame has an "employee" column, and automatically joins using this column as a key. The result of the merge is a new DataFrame that combines the information from the two inputs. Notice that the order of entries in each column is not necessarily maintained: in this case, the order of the "employee" column differs between df1 and df2, and the pd.merge() function correctly accounts for this. Additionally, keep in mind that the merge in general discards the index, except in the special case of merges by index 

#### Many-to-one joins

Joins in which the two key columns contains duplicate entires. 
- The resulting DataFrame will preserver duplicate entries as possible

In [75]:
df4 = pd.DataFrame({'group': ['Accounting', 'Engineering', 'HR'],
                    'supervisor': ['Carly', 'Guido', 'Steve']})
display('df3', 'df4', 'pd.merge(df3, df4)')

df3
  employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014

df4
         group supervisor
0   Accounting      Carly
1  Engineering      Guido
2           HR      Steve

pd.merge(df3, df4)
  employee        group  hire_date supervisor
0      Bob   Accounting       2008      Carly
1     Jake  Engineering       2012      Guido
2     Lisa  Engineering       2004      Guido
3      Sue           HR       2014      Steve

#### Many-to-many joins
If the key column in both the left and right array contains duplicates, then the result is a many-to-many merge.

In [76]:
df5 = pd.DataFrame({'group': ['Accounting', 'Accounting',
                              'Engineering', 'Engineering', 'HR', 'HR'],
                    'skills': ['math', 'spreadsheets', 'coding', 'linux',
                               'spreadsheets', 'organization']})
display('df1', 'df5', "pd.merge(df1, df5)")

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
,group,skills
0,Accounting,math
1,Accounting,spreadsheets
2,Engineering,coding
3,Engineering,linux
4,HR,spreadsheets


#### Specifying the name of the key column using the ``on`` keyword.

In [77]:
display('df1', 'df2', "pd.merge(df1, df2, on='employee')")

df1
  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR

df2
  employee  hire_date
0     Lisa       2004
1      Bob       2008
2     Jake       2012
3      Sue       2014

pd.merge(df1, df2, on='employee')
  employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014

#### left_on and right_on keywords

In [78]:
df3 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'salary': [70000, 80000, 120000, 90000]})
display('df1', 'df3', 'pd.merge(df1, df3, left_on="employee", right_on="name")')

df1
  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR

df3
   name  salary
0   Bob   70000
1  Jake   80000
2  Lisa  120000
3   Sue   90000

pd.merge(df1, df3, left_on="employee", right_on="name")
  employee        group  name  salary
0      Bob   Accounting   Bob   70000
1     Jake  Engineering  Jake   80000
2     Lisa  Engineering  Lisa  120000
3      Sue           HR   Sue   90000

#### Dropping the redundant column using drop() method

In [79]:
pd.merge(df1, df3, left_on="employee", right_on="name").drop('name', axis=1)

,employee,group,salary
0,Bob,Accounting,70000
1,Jake,Engineering,80000
2,Lisa,Engineering,120000
3,Sue,HR,90000


#### Merging on an index using left_index and right_index keywords

In [80]:
df1a = df1.set_index('employee')
df2a = df2.set_index('employee')
display('df1a', 'df2a')

,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR
,hire_date
employee,
Lisa,2004
Bob,2008
Jake,2012


In [81]:
display('df1a', 'df2a',
        "pd.merge(df1a, df2a, left_index=True, right_index=True)")

,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR
,hire_date
employee,
Lisa,2004
Bob,2008
Jake,2012


#### Using join() method for convenience
- Using join() results in a merge that defaults to joining on indices

In [82]:
display('df1a', 'df2a', 'df1a.join(df2a)')

,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR
,hire_date
employee,
Lisa,2004
Bob,2008
Jake,2012


#### Mixing indices and columns for merge. 

In [83]:
display('df1a', 'df3', "pd.merge(df1a, df3, left_index=True, right_on='name')")

df1a
                group
employee             
Bob        Accounting
Jake      Engineering
Lisa      Engineering
Sue                HR

df3
   name  salary
0   Bob   70000
1  Jake   80000
2  Lisa  120000
3   Sue   90000

pd.merge(df1a, df3, left_index=True, right_on='name')
         group  name  salary
0   Accounting   Bob   70000
1  Engineering  Jake   80000
2  Engineering  Lisa  120000
3           HR   Sue   90000

## Aggregation and Grouping

In [84]:
import seaborn as sns
planets = sns.load_dataset('planets')
planets.shape

(1035, 6)

In [85]:
planets.head()

,method,number,orbital_period,mass,distance,year
0,Radial Velocity,1,269.300,7.10,77.40,2006
1,Radial Velocity,1,874.774,2.21,56.95,2008
2,Radial Velocity,1,763.000,2.60,19.84,2011
3,Radial Velocity,1,326.030,19.40,110.62,2007
4,Radial Velocity,1,516.220,10.50,119.47,2009


### Simple Aggregations

In [86]:
rng = np.random.RandomState(42)
ser = pd.Series(rng.rand(5))
ser

0    0.374540
1    0.950714
2    0.731994
3    0.598658
4    0.156019
dtype: float64

In [87]:
ser.sum()

2.811925491708157

In [88]:
ser.mean()

0.5623850983416314

In [89]:
df = pd.DataFrame({'A': rng.rand(5),
                   'B': rng.rand(5)})
df

,A,B
0,0.155995,0.020584
1,0.058084,0.969910
2,0.866176,0.832443
3,0.601115,0.212339
4,0.708073,0.181825


For a DataFrame, by default the aggregates return results within each column:

In [90]:
df.mean()

A    0.477888
B    0.443420
dtype: float64

Using axis keyword

In [91]:
df.mean(axis='columns')

0    0.088290
1    0.513997
2    0.849309
3    0.406727
4    0.444949
dtype: float64

In [92]:
planets.dropna().describe()

,number,orbital_period,mass,distance,year
count,498.00000,498.000000,498.000000,498.000000,498.000000
mean,1.73494,835.778671,2.509320,52.068213,2007.377510
std,1.17572,1469.128259,3.636274,46.596041,4.167284
min,1.00000,1.328300,0.003600,1.350000,1989.000000
25%,1.00000,38.272250,0.212500,24.497500,2005.000000
50%,1.00000,357.000000,1.245000,39.940000,2009.000000
75%,2.00000,999.600000,2.867500,59.332500,2011.000000
max,6.00000,17337.500000,25.000000,354.000000,2014.000000


#### Built-in Pandas aggregations  


| Aggregation              | Description                     |
|--------------------------|---------------------------------|
| ``count()``              | Total number of items           |
| ``first()``, ``last()``  | First and last item             |
| ``mean()``, ``median()`` | Mean and median                 |
| ``min()``, ``max()``     | Minimum and maximum             |
| ``std()``, ``var()``     | Standard deviation and variance |
| ``mad()``                | Mean absolute deviation         |
| ``prod()``               | Product of all items            |
| ``sum()``                | Sum of all items                |


In [93]:
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'data': range(6)}, columns=['key', 'data'])
df

,key,data
0,A,0
1,B,1
2,C,2
3,A,3
4,B,4
5,C,5


In [94]:
df.groupby('key')

In [95]:
df.groupby('key').sum()

,data
key,
A,3
B,5
C,7
